# Construction du Dataset

## Imports

In [5]:
from lyricsgenius import Genius
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
import os

## Déclaration des tokens pour se connecter aux API

In [6]:
username=os.environ["SPOTIFY_USERNAME"]
cid= os.environ["SPOTIFY_CID"]
secret=os.environ["SPOTIFY_SECRET"]
g_id = os.environ["GENIUS_ID"]
g_secret = os.environ["GENIUS_SECRET"]
token = os.environ["GENIUS_TOKEN"]

genius = Genius(token)
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
import os

print(os.environ["SPOTIFY_SECRET"])

## Fonctions

In [20]:
#insert output dataframe from the get_album_tracks function
def get_track_info(df):
    danceability = []
    energy = []
    key = []
    loudness = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    liveness = []
    valence = []
    tempo = []
    n=0
    for i in df['uri']:
        for x in sp.audio_features(tracks=[i]):
            n=n+1
            print(str(n)+"/"+str(len(df)+1))
            try:
                danceability.append(x['danceability'])
                energy.append(x['energy'])
                key.append(x['key'])
                loudness.append(x['loudness'])
                speechiness.append(x['speechiness'])
                acousticness.append(x['acousticness'])
                instrumentalness.append(x['instrumentalness'])
                liveness.append(x['liveness'])
                valence.append(x['valence'])
                tempo.append(x['tempo'])
                
            except:
                danceability.append(None)
                energy.append(None)
                key.append(None)
                loudness.append(None)
                speechiness.append(None)
                acousticness.append(None)
                instrumentalness.append(None)
                liveness.append(None)
                valence.append(None)
                tempo.append(None)
    df2 = pd.DataFrame({
    'danceability':danceability,
    'energy':energy,
    'key':key,
    'loudness':loudness,
    'speechiness':speechiness,
    'acousticness':acousticness,
    'instrumentalness':instrumentalness,
    'liveness':liveness,
    'valence':valence,
    'tempo':tempo})
    
    return df2

In [16]:
def get_song_infos(df1):
    producer_artist=[]
    writer_artist=[]
    music_label=[]
    distributor=[]
    number_views_genius=[]
    language=[]
    lyrics=[]
    n=0
    for i in range(len(df1)):
        n=n+1
        print(str(n)+"/"+str(len(df1)+1))
        try:
            test = genius.search_song(df1['track'].loc[i], df1.loc[i]['artist'][0])
            song = genius.song(test.id).get('song')
            try:
                producer_artist.append([x.get('name') for x in song.get('producer_artists')])
                
            except:
                producer_artist.append(None)
            
            try:
                writer_artist.append([x.get('name') for x in song.get('writer_artists')])
            
            except:
                writer_artist.append(None)
            
            try:
                music_label.append([x.get('name') for x in song.get('custom_performances')[6].get('artists')])
            
            except:
                music_label.append(None)
                
            try:
                distributor.append([x.get('name') for x in song.get('custom_performances')[7].get('artists')])
            
            except:
                distributor.append(None)
            
            try:
                number_views_genius.append(song.get('stats').get('pageviews'))
            
            except:
                number_views_genius.append(None)
            
            try:
                language.append(song.get('language'))
            
            except:
                language.append(None)
            try:
                lyrics.append(test.lyrics)
            except:
                lyrics.append(None)
        except:
            producer_artist.append(None)
            writer_artist.append(None)
            music_label.append(None)
            distributor.append(None)
            number_views_genius.append(None)
            language.append(None)
            lyrics.append(None)
    
            
    df1['producer_artist']=producer_artist
    df1['writer_artist']=writer_artist
    df1['music_label'] = music_label
    df1['distributor'] = distributor
    df1['number_views_genius'] = number_views_genius
    df1['language'] = language
    df1['lyrics'] = lyrics
    return df1

In [1]:
def get_playlist_tracks(uri_playlist, theme):
    one = sp.playlist_tracks(uri_playlist, limit=50, offset=0, market='US')
    df=pd.DataFrame(one)
    dictionnaire=df['items'].to_dict()
    artist=[]
    uri_song=[]
    duration_ms=[]
    track=[]
    release_date=[]
    album=[]
    popularity=[]
    for element in dictionnaire.values(): 
        artist.append([x.get('name') for x in element.get('track').get('artists')])
        uri_song.append(element.get('track').get('uri'))
        duration_ms.append(element.get('track').get('duration_ms'))
        track.append(element.get('track').get('name'))
        release_date.append(element.get('track').get('album').get('release_date'))
        popularity.append(element.get('track').get('popularity'))
        album.append(element.get('track').get('album').get('name'))
        
    theme=[theme]*len(album)
    df1=pd.DataFrame({
        'uri':uri_song,
        'track':track,
        'album':album,
        'artist':artist,
        'categorie':theme,
        'duration_ms':duration_ms,
        'release_date':release_date,
        'popularity': popularity,
        'theme':None,
    })
    return df1

In [7]:
def merge_frames(df1, df2):
    df3 = df1.merge(df2, left_index= True, right_index= True)
    return df3

## Create dataframe of playlist

In [8]:
dict_categories=sp.categories().get('categories')
id_categorie=[]
name_categorie=[]
for element in dict_categories.get('items'):
    id_categorie.append(element.get('id'))
    name_categorie.append(element.get('name'))

all_dict_playlist=[]
all_playlist=[]
for categorie in id_categorie[:19]:
    all_dict_playlist.append(sp.category_playlists(categorie).get('playlists').get('items'))
for element in all_dict_playlist:
    all_playlist.append([x.get('uri') for x in element if x is not None])
    
playlist_top_lists=all_playlist[0]
playlist_summer=all_playlist[1]
playlist_hip_hop=all_playlist[2]
playlist_pop=all_playlist[3]
playlist_french_pop=all_playlist[4]
playlist_mood=all_playlist[5]
playlist_dance_electronic=all_playlist[6]
playlist_rock=all_playlist[7]
playlist_latin=all_playlist[8]
playlist_party=all_playlist[9]
playlist_r_et_b=all_playlist[10]
playlist_chill=all_playlist[11]
playlist_afro=all_playlist[12]
playlist_netflix=all_playlist[13]
playlist_workout=all_playlist[14]
playlist_sleep=all_playlist[15]
playlist_in_the_car=all_playlist[16]
playlist_dancehall_zouk=all_playlist[17]
playlist_decades=all_playlist[18]

first_playlists=[]
first_playlists.append([x[:2] for x in all_playlist])
all_df_playlist=[]
for i, playlist in enumerate(first_playlists[0]):
    all_df_playlist.append(pd.DataFrame({'playlist':playlist, 'theme': name_categorie[i]}))
    
all_df=[]
for i in range(len(all_df_playlist)):
    for j in range(len(all_df_playlist[i])):
        try:
            all_df.append(get_playlist_tracks(all_df_playlist[i].loc[j].values[0], list(all_df_playlist[i]['theme'].unique())[0]))
        except:
            print(all_df_playlist[i].loc[j].values[0])
            print(list(all_df_playlist[i]['theme'].unique())[0])
df=pd.concat(all_df, ignore_index=True)

## Dataframe infos

In [ ]:
df_song_info = get_song_infos(df)
df_song_info

## Fusion des dataframes

In [22]:
df_song_info2 = get_track_info(df_song_info)
df_final = merge_frames(df_song_info,df_song_info2)
df_final.categorie.unique()

## Filtrage des dataframes et concaténation

Dans le cadre de ce projet, nous avons décidé de restreindre notre ontologie aux morceaux français. Afin d'avoir un dataset homogène, on extrait 10 morceaux de chacun de nos genres.

In [34]:
categories = ["Top Lists", "Summer", "Hip-Hop", "Pop", "French Pop", "Mood", "Dance/Electronic", "Rock", "latin", "Party", "R&B", "Chill", "Afro", "Netflix", "Workout", "Sleep", "In the car", "Dancehall/Zouk", "Decades"]

dfs = [df_final.query(f'language=="fr" and categorie == "{categorie}"')[:10] for categorie in categories]

music_id=pd.concat(dfs, ignore_index=True)
music_id.to_json("music_id.json", orient="records")